<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/DIRE_preprocess_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ZhendongWang6/DIRE.git

Cloning into 'DIRE'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 63 (delta 7), reused 59 (delta 6), pack-reused 0
Receiving objects: 100% (63/63), 1.95 MiB | 5.78 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt

--2024-03-30 14:56:05--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.209.18.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.209.18.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2211383297 (2.1G) [application/octet-stream]
Saving to: ‘256x256_diffusion_uncond.pt’

256x256_diffusion_u 100%[===================>]   2.06G  11.0MB/s    in 3m 22s  

2024-03-30 14:59:28 (10.4 MB/s) - ‘256x256_diffusion_uncond.pt’ saved [2211383297/2211383297]



In [4]:
import os
os.makedirs('DIRE/models', exist_ok=True)
import shutil
shutil.move('/content/256x256_diffusion_uncond.pt', 'DIRE/models')


'DIRE/models/256x256_diffusion_uncond.pt'

In [5]:
!pip install ftfy -q
!pip install natsort -q
!pip install tensorboardX -q
!pip install blobfile -q
!pip install mpi4py -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
#@title Unzip dataset
#!cp /content/drive/MyDrive/DATASETS/RealFakeDB_tiny.zip -d /content
dataset_zip_file = '/content/drive/MyDrive/DATASETS/RealFakeDB_tiny.zip' #@param {type:"string"}
unzip_to = 'DIRE' #@param {type:"string"}
!unzip -q $dataset_zip_file -d $unzip_to

In [7]:
%cd DIRE

/content/DIRE


#Process for test set
(We need to remove all lines related to multi-GPU configuration in compe_dire.py before run)

#Process for train set

In [8]:
import os
image_list = []
extensions = ['.jpg', '.png', '.gif', '.bmp', '.webp']  # Add more extensions as needed
def getname(base_dir):
    for dirpath, dirnames, filenames in os.walk(base_dir):
        if len(filenames) > 0:
            imgs = [filename for filename in filenames if any(ext in filename for ext in extensions)]
            image_list.extend(imgs)
    return image_list


In [9]:
image_list=[]
image_list = getname('/content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny/dire_test')


In [16]:
len(image_list)

992

In [ ]:
#@title Remove all items existed in $image_list before run
data_dir = '/content/DIRE/RealFakeDB_tiny' #@param {type:"string"}
c=0
for dirpath, dirnames, filenames in os.walk(data_dir):
    if len(filenames) >0:
        for file in filenames:
            if file in image_list:
                c+=1
                full_name = os.path.join(dirpath, file)
                os.remove(full_name)
                print('removed {}:{}:'.format(c,full_name))

samples = len(os.listdir(data_dir + '/0_real')) + len(os.listdir(data_dir + '/1_fake'))
print('num sample remain :', samples)

In [33]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# edit add wepb to        if "." in entry and ext.lower() in ["jpg", "jpeg", "png", "gif", "webp"]: in image_dataset.py gaussioan_diffusion
!python guided-diffusion/compute_dire.py \
--model_path models/256x256_diffusion_uncond.pt \
--attention_resolutions 32,16,8 \
--class_cond False \
--diffusion_steps 1000 --dropout 0.1 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 \
--num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
--images_dir /content/DIRE/RealFakeDB_tiny/test/1_fake \
--recons_dir /content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny/recons_test/1_fake \
--dire_dir /content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny/dire_test/1_fake \
--batch_size 16 --num_samples 1016  --timestep_respacing ddim20 --use_ddim True \
--has_subfolder False

Logging to /content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny/recons_test/1_fake
Namespace(images_dir='/content/DIRE/RealFakeDB_tiny/test/1_fake', recons_dir='/content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny/recons_test/1_fake', dire_dir='/content/drive/MyDrive/DATASETS/DIRE/RealFakeDB_tiny/dire_test/1_fake', clip_denoised=True, num_samples=1016, batch_size=16, use_ddim=True, model_path='models/256x256_diffusion_uncond.pt', real_step=0, continue_reverse=False, has_subfolder=False, image_size=256, num_channels=256, num_res_blocks=2, num_heads=4, num_heads_upsample=-1, num_head_channels=64, attention_resolutions='32,16,8', channel_mult='', dropout=0.1, class_cond=False, use_checkpoint=False, use_scale_shift_norm=True, resblock_updown=True, use_fp16=True, use_new_attention_order=False, learn_sigma=True, diffusion_steps=1000, noise_schedule='linear', timestep_respacing='ddim20', use_kl=False, predict_xstart=False, rescale_timesteps=False, rescale_learned_sigmas=False)
have created mo